In [ ]:
# Fix working directory and Python path to find src module from scripts directory
import sys
import os

# Get current working directory and manually remove "scripts" if present
cwd = os.getcwd()
print(f"Original working directory: {cwd}")

# If we're in scripts directory, change to the parent directory
if cwd.endswith("scripts"):
    project_root = os.path.dirname(cwd)
    os.chdir(project_root)
    print(f"Changed working directory to: {os.getcwd()}")
else:
    project_root = cwd
    print(f"Already in project root: {project_root}")

# Add project root to Python path if not already present
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Final working directory: {os.getcwd()}")
print(f"Python path updated. First few entries: {sys.path[:3]}")

## Imports and Variables

In [ ]:
from src.data_structures import ExperimentConfig, ControlConfig, TreatmentConfig
from src.inspect_helpers.tasks import injection_consistency_and_recognition
from src.inspect_helpers.datasets import ROW_INDEX_KEY
from src.inspect_helpers.scorers import custom_match, custom_prompt_criterion_mgf
from src.inspect_helpers.utils import collect_logs_by_model, get_validated_logs_by_model
from inspect_ai.log import EvalLog, list_eval_logs, read_eval_log
from inspect_ai.model import (
    Model,
    ModelAPI,
    GenerateConfig,
    anthropic,
    ollama,
    get_model,
)
from inspect_ai import eval, eval_async
import pandas as pd
import os

EXPERIMENT_NAME = "wikihow_summary_injection_PT_v2_scorer_v2_sonnet_4"
CONTROL_LOG_DIR = f"logs/{EXPERIMENT_NAME}/control"
TREATMENT_LOG_DIR = f"logs/{EXPERIMENT_NAME}/treatment"

START_IDX = 0
END_IDX = 20


MODELS = [
    "anthropic/claude-sonnet-4-20250514",
    #"anthropic/claude-3-5-haiku-20241022",
    #"ollama/gemma3:1b-it-q8_0",
    # "ollama/llama3.2:1b-instruct-q8_0"
]

#SCORING_MODEL= "together/Qwen/Qwen3-235B-A22B-Instruct-2507-tput"
SCORING_MODEL= "anthropic/claude-3-5-haiku-20241022"

islocal = {
    "ollama": True,
    "together": False,
    "anthropic": False,
    "google": False,
}


def split_provider_and_model(model: str) -> str:
    return model.split("/")[0], model.split("/")[1]


PROMPT_TEMPLATE_ARGS = {
    "summary_adjectives": "very long and detailed, single-paragraph",
}

BATCH_SIZE_LOCAL = 4
MAX_CONNECTIONS_API = 100

LIMIT = 1


In [ ]:
def windows_safe_path(path: str) -> str:
    return path.replace(":", "_")

In [ ]:
from src.data.treatments.wikisum_utils import get_WikiSum, get_WikiSum_random

df = get_WikiSum(
    START_IDX,
    END_IDX,
    save_path="data/",
    splits=["train"],
    columns=["id", "title", "text"],
)
df

## Control Evaluation

## Make CSVs from the control eval logs

## Applying treatments to csv datasets

## Treatment Evaluations

# Summarising results

In [1]:
from inspect_ai.analysis import evals_df
CONTROL_LOG_DIR = f"logs/wikihow_summary_injection_PT_v2_scorer_v2_sonnet_4/control" 
TREATMENT_LOG_DIR = f"logs/wikihow_summary_injection_PT_v2_scorer_v2_sonnet_4/treatment"

control_evals_df = evals_df(CONTROL_LOG_DIR)
treatment_evals_df = evals_df(TREATMENT_LOG_DIR)
control_evals_df.columns

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'c:/Users/jesse/Documents/Code_Projects/Python/injection-recognition/scripts/logs/wikihow_summary_injection_PT_v2_scorer_v2_sonnet_4/control'

In [ ]:
import pandas as pd
from inspect_ai.log import list_eval_logs
from inspect_ai.analysis import evals_df, prepare

control_logs = list_eval_logs(CONTROL_LOG_DIR, filter=lambda log: log.status == "success")
treatment_logs = list_eval_logs(TREATMENT_LOG_DIR, filter=lambda log: log.status == "success")

control_evals_df = evals_df(control_logs)
treatment_evals_df = evals_df(treatment_logs)
    
control_and_treatments_df = pd.concat([control_evals_df, treatment_evals_df])

control_and_treatments_df.columns

In [ ]:
control_and_treatments_df.task_arg_csv_file_path

Axes of interest:

Bar chart:
- Model
- Model provider (pattern)
- Treatment type (Seperate plots)
- Treatment strength (h_concat)
- Injection length (0 for control) (v_concat)
- Whether injection? Score & stderr (y)
- What injection? Score & stderr 

1. Filter to status = "success"
2. make separate columns for injection length from task_arg_treatment_col (0 for control evals)
3. make separate columns for treatment strength from task_arg_treatment_col
4. Make a column for what injection? from task_arg_csv_file_path
5. Make a column for whether injection? from injection length



In [ ]:
from src.analyzer import Analyzer

evals_analyzer = Analyzer(control_and_treatments_df)

def get_injection_length(treatment_col : str | None) -> int:
    if treatment_col is None or pd.isna(treatment_col):
        return 0
    return int(treatment_col.split("IL")[1].split("_")[0])

def get_treatment_strength(treatment_col : str | None) -> str | None:
    if treatment_col is None or pd.isna(treatment_col):
        return None
    return treatment_col.split("_")[1]

def get_treatment_type(file_path : str | None) -> str | None:
    if file_path is None or pd.isna(file_path):
        return None
    file_name = file_path.split("/")[-1]
    if file_name.startswith("dataset_") and file_name.endswith("injected.csv"):
        return file_name.split("_")[1]
    return None

evals_analyzer.add_column(
    column_name="has_treatment",
    column_spec = {
        "task_arg_treatment_col": lambda x : x is not None and not pd.isna(x)
    }
)

evals_analyzer.add_column(
    column_name="injection_length",
    column_spec = {
        "task_arg_treatment_col": get_injection_length
    }
)

evals_analyzer.add_column(
    column_name="treatment_strength",
    column_spec = {
        "task_arg_treatment_col": get_treatment_strength
    }
)

evals_analyzer.add_column(
    column_name="treatment_type",
    column_spec = {
        "task_arg_csv_file_path": get_treatment_type
    }
)






# Replace S0 with "light" and S4 with "heavy" in treatment_strength column
evals_analyzer.df['treatment_strength'] = evals_analyzer.df['treatment_strength'].replace({
    'S0': 'light',
    'S4': 'heavy'
})

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv(r'data\wikihow_summary_injection_PT_v2_scorer_v2\anthropic_claude-3-5-haiku-20241022\dataset_typo_rates_injected.csv')

# Display basic info about the DataFrame
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
df.head()

In [ ]:
from src.visualizer import VisualisationConfig, visualize
import altair as alt

visualize(
    evals_analyzer.df,
    VisualisationConfig(
        plot_fn=alt.Chart.mark_bar,
        x_category="model",
        y_category="mean(score_custom_match_accuracy)",
        h_concat_category="treatment_type",
    ),
)

In [ ]:
from src.visualizer import VisualisationConfig, visualize
import altair as alt

visualize(
    evals_analyzer.df,
    VisualisationConfig(
        plot_fn=alt.Chart.mark_bar,
        x_category="model",
        y_category="mean(score_custom_prompt_criterion_mgf_accuracy)",
        h_concat_category="treatment_type",
        v_concat_category="treatment_strength",
    ),
)

In [ ]:
from src.visualizer import VisualisationConfig, visualize
import altair as alt

visualize(
    evals_analyzer.df,
    VisualisationConfig(
        plot_fn=alt.Chart.mark_bar,
        x_category="treatment_strength",
        y_category="mean(score_custom_prompt_criterion_mgf_accuracy)",
        h_concat_category="treatment_type",
        v_concat_category="model",
    ),
)

In [ ]:
visualize(
    evals_analyzer.df,
    VisualisationConfig(
        plot_fn=alt.Chart.mark_bar,
        x_category="injection_length",
        y_category="mean(score_custom_prompt_criterion_mgf_accuracy)",
        h_concat_category="treatment_type",
        v_concat_category="model",
    ),
)

In [ ]:
evals_analyzer.df.columns

In [ ]:
evals_analyzer.df["injection_length"]